In [32]:
import pandas as pd

df = pd.read_csv('/Users/skog/Documents/1-2023-autumn/school/TDT4259-applied-data-science/data.csv')

from datetime import datetime

def convert_to_formats(timestamp_str):
    # Parse the given timestamp string
    dt = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')

    # Convert to nanosecond UNIX epoch format
    epoch_ns = int(dt.timestamp() * 1e9)

    # Convert to RFC3339 format (it's essentially an ISO8601 format which is what datetime uses by default for 'isoformat' method)
    rfc3339_format = dt.isoformat()

    return rfc3339_format

df['time'] = df['time'].apply(convert_to_formats)

df = df[df["location"] == "trondheim"]


df.head()

,time,location,consumption,temperature
4,2022-04-07T21:00:00,trondheim,1.970098,-2.8
9,2022-04-07T22:00:00,trondheim,1.839443,-3.3
14,2022-04-07T23:00:00,trondheim,1.747675,-3.8
19,2022-04-08T00:00:00,trondheim,1.703176,-4.2
24,2022-04-08T01:00:00,trondheim,1.694300,-4.1


In [27]:
# load csv file
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import BATCH

from influxdb_client import Point, InfluxDBClient, WriteOptions
from influxdb_client.client.write_api import WriteType




os.environ['INFLUXDB_TOKEN']='7l9z4vyVzXLN9G5MT7asSw7Qry-8ckJLw-EYkGBHYZVOs9jKxUv0H07yx66VahIT9g9BUTGHutFXCrgt3CmhcA=='

token = os.environ.get("INFLUXDB_TOKEN")
org = "admin"
url = "http://localhost:8086"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org, debug=True)

bucket="admin"

write_api = write_client.write_api(write_options=WriteType.batching, batch_size=50_000, flush_interval=10_000)
   
# for value in range(5):
#   point = (
#     Point("measurement2")
#     .tag("tagname1", "tagvalue1")
#     .field("field1", value)
#   )

#   write_api.write(bucket=bucket, org="admin", record=point)
#   time.sleep(1) # separate points by 1 second

# cols = ["name, calories"]

# # df = df[cols]
# df = df[["name", "calories"]]

# df

# to csv
# df.to_csv('/Users/skog/Downloads/cereal1.csv', index=False)

In [33]:
#  only keep 100 rows of df

# write all locations to bucket "test"
bucket = "test"

for index, row in df.iterrows():
  point = (
    Point("measurement2")
    .tag("location",row["location"])
    .field("consumption", row["consumption"])
    .field("temperature", row["temperature"])
    .time(row["time"], WritePrecision.NS)
  )

  write_api.write(bucket=bucket, org="admin", record=point)

KeyboardInterrupt: 

In [30]:
query_api = write_client.query_api()

# query = """from(bucket: "admin")
#  |> range(start: -10m)
#  |> filter(fn: (r) => r._measurement == "measurement1")"""

query = """from(bucket: "admin") |> range(start: 2010-01-01T00:00:00Z, stop: 2030-01-01T00:00:00Z) |> filter(fn: (r) => r["_measurement"] == "measurement2") |> count() |> yield(name: "count")"""
tables = query_api.query(query, org="admin")

for table in tables:
  for record in table.records:
    print(record)


FluxRecord() table: 0, {'result': 'count', 'table': 0, '_start': datetime.datetime(2010, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2030, 1, 1, 0, 0, tzinfo=tzutc()), '_value': 100, '_field': 'consumption', '_measurement': 'measurement2', 'tagname1': 'tagvalue1'}
FluxRecord() table: 1, {'result': 'count', 'table': 1, '_start': datetime.datetime(2010, 1, 1, 0, 0, tzinfo=tzutc()), '_stop': datetime.datetime(2030, 1, 1, 0, 0, tzinfo=tzutc()), '_value': 100, '_field': 'temperature', '_measurement': 'measurement2', 'tagname1': 'tagvalue1'}


In [10]:
query_api = write_client.query_api()

query = """from(bucket: "admin")
 |> range(start: -10m)
 |> filter(fn: (r) => r._measurement == "measurement1")"""
tables = query_api.query(query, org="admin")

for table in tables:
  for record in table.records:
    print(record)


FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 10, 23, 21, 56, 51, 221571, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 10, 23, 22, 6, 51, 221571, tzinfo=tzutc()), '_time': datetime.datetime(2023, 10, 23, 22, 6, 16, 98448, tzinfo=tzutc()), '_value': 0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 10, 23, 21, 56, 51, 221571, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 10, 23, 22, 6, 51, 221571, tzinfo=tzutc()), '_time': datetime.datetime(2023, 10, 23, 22, 6, 17, 137497, tzinfo=tzutc()), '_value': 1, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 10, 23, 21, 56, 51, 221571, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 10, 23, 22, 6, 51, 221571, tzinfo=tzutc()), '_time': datetime.datetime(2

In [11]:
query_api = write_client.query_api()

query = """from(bucket: "admin")
  |> range(start: -10m)
  |> filter(fn: (r) => r._measurement == "measurement1")
  |> mean()"""
tables = query_api.query(query, org="admin")

for table in tables:
    for record in table.records:
        print(record)

FluxRecord() table: 0, {'result': '_result', 'table': 0, '_start': datetime.datetime(2023, 10, 23, 21, 57, 19, 410666, tzinfo=tzutc()), '_stop': datetime.datetime(2023, 10, 23, 22, 7, 19, 410666, tzinfo=tzutc()), '_value': 2.0, '_field': 'field1', '_measurement': 'measurement1', 'tagname1': 'tagvalue1'}
